In [3]:
!pip install pennylane
!pip install pennylane-qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.3/341.3 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5

In [10]:
import pennylane as qml
import numpy as np

dev = qml.device('qiskit.aer', wires=2)

# start the quantum circuit
@qml.qnode(dev)
def quantum_circuit(weights, x):
    #input transformation
    qml.BasisState(x, wires=[0, 1])

    # we call it parametrized rotation (gates layer fi plasset classical layer)
    qml.Rot(weights[0], weights[1], weights[2], wires=0)
    qml.Rot(weights[3], weights[4], weights[5], wires=1)
    qml.CNOT(wires=[0, 1])

    qml.Rot(weights[6], weights[7], weights[8], wires=0)
    qml.Rot(weights[9], weights[10], weights[11], wires=1)
    qml.CNOT(wires=[0, 1])

    return qml.expval(qml.PauliZ(1))

def cost(weights, X, Y):
    predictions = [quantum_circuit(weights, x) for x in X]
    return np.mean((np.array(predictions) - Y) ** 2)

X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([0, 1, 1, 0])

#random weights
weights = np.random.rand(12) * 2 * np.pi
learning_rate = 0.1
for epoch in range(100):
    #simulating the backpropagation since we don't have a predefined way to do so
    grad = np.zeros_like(weights)
    for i in range(len(weights)):
        weights[i] += 0.01
        loss_plus = cost(weights, X, Y)
        weights[i] -= 0.02
        loss_minus = cost(weights, X, Y)
        grad[i] = (loss_plus - loss_minus) / 0.02
        weights[i] += 0.01

    weights -= learning_rate * grad
    if epoch % 10 == 0:
        print(f"epoch {epoch}: loss = {cost(weights, X, Y)}")

print("Trained weights:", weights)

epoch 0: loss = 0.5929841995239258
epoch 10: loss = 0.6849908828735352
epoch 20: loss = 0.5359926223754883
epoch 30: loss = 0.3718242645263672
epoch 40: loss = 0.4421653747558594
epoch 50: loss = 0.3833293914794922
epoch 60: loss = 0.4204854965209961
epoch 70: loss = 0.5590343475341797
epoch 80: loss = 0.5588846206665039
epoch 90: loss = 0.5376672744750977
Trained weights: [ 3.66921553  5.32173824  4.26625731  3.85233577  6.33239046  4.95213784
  4.36618666  4.164893    5.60764882  4.60624928 -1.43343594  2.96214707]


In [11]:
def predict(weights, x):
    return quantum_circuit(weights, x)

for input_data in X:
    prediction = predict(weights, input_data)
    binary_prediction = 1 if prediction > 0.5 else 0
    print(f"{input_data} : {binary_prediction}")

[0 0] : 0
[0 1] : 1
[1 0] : 0
[1 1] : 0
